In [1]:
pip install transformers

In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `mistralai` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `mistrala

In [ ]:
import csv
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformers import BitsAndBytesConfig
import pandas as pd

quantization_config = BitsAndBytesConfig(load_in_8bit=True)
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# Function to process one sentence at a time
def extract_named_entities_batch(sentences):
    input_texts = [
        f"Extract only the person's full name (first, middle, last) from the following text. "
        "Do not include numbers, addresses, or locations.\n\n"
        "Examples:\n"
        "Input: jose m, fair, 6313 marconi st , huntington park\nOutput: Jose M, Fair\n"
        "Input: maria h, essgi , 11341 ,avenida de los lobos, apartment se ,san diego\nOutput: Maria H, Essgi\n"
        f"Input: {sentence}\nOutput:"
        for sentence in sentences
    ]

    # Tokenize with padding and move to the correct device (CPU)
    inputs = tokenizer(input_texts, padding=True, truncation=True, return_tensors="pt").to("cpu")

    # Generate results
    outputs = model.generate(
        inputs.input_ids,
        max_new_tokens=5,
        do_sample=False
    )

    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

# Process one sentence at a time (avoid large batch sizes)
gemma_results = []
for sentence in sentences:
    batch_results = extract_named_entities_batch([sentence])  # Process one sentence at a time
    gemma_results.append(batch_results[0])  # Append result from each sentence

# Save Gemma results to CSV
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Record Number", "Original Sentence", "Extracted Names (Gemma)"])

    for i, (sentence, gemma_name) in enumerate(zip(sentences, gemma_results), start=1):
        writer.writerow([i, sentence, gemma_name])

print(f"\n✅ Extraction complete! Results saved to '{output_file}'")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

NameError: name 'sentences' is not defined

In [ ]:
def extract_named_entities_batch(sentences):
    input_text =[
    "Extract and return only the person's full name (first, middle, last) from the given address record. "
    "Do not include numbers, addresses, or locations. Respond with only the name and nothing else.\n\n"
    "Use the same capitalization and punctuation style as shown.\n\n"
        "Examples:\n"
        "Input: jose m, fair, 6313 marconi st , huntington park\nOutput: Jose M, Fair\n"
        "Input: maria h, essgi , 11341 ,avenida de los lobos, apartment se ,san diego\nOutput: Maria H, Essgi\n"
        "Input: allne ernesto, g , 7359, ircle dr , lady lake\nOutput: Allne Ernesto, G\n"
        "Input: maria , migenes , 1428, apartado, san pedro\nOutput: Maria, Migenes\n"
        "Input: roberto bruce, wtason, 570, mooney rd ne, fort walton beach\nOutput: Roberto Bruce, Wtason\n"
        "Input: marcelo guy, wilcox, 540, florida club blvd no112, st augustine\nOutput: Marcelo Guy, Wilcox\n"
        "Input: villareal ,1909,dr martin luther king jr way,haines city,martin luther king jr\nOutput: Villareal\n"
        "Input: b, ventura ,2616,canada blvd apt 204,glendle,glendle\nOutput: B, Ventura\n"
        "Input: leslie, unn, 326, staples ave, san jose\nOutput: Leslie, Unn\n\n"
        f"Input: {sentence}\nOutput:"
        for sentence in sentences
    ]
   # Tokenize with padding and move to the correct device (CPU)
    inputs = tokenizer(input_text, padding=True, truncation=True, max_length=512,return_tensors="pt").to("cuda")
    outputs = model.generate(
    inputs.input_ids,
    max_new_tokens=3,  # Reduce token limit
    do_sample=False
    )

    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]


# Function to extract names with regex (to be combined with Gemma)
def extract_name(sentence):
    # Normalize commas and spaces
    sentence = re.sub(r"\s*,\s*", ", ", sentence.strip())

    # Extract the name part (before the first number or street keyword)
    match = re.match(r"([a-zA-Z\s]+(?:,\s*[a-zA-Z\s]+)*)", sentence)

    if match:
        name = match.group(1)
        # Capitalize first letters of each name
        return " ".join(word.capitalize() for word in name.split(", "))
    return ""

# Read CSV safely and fill NaN values
input_file = "S12PX.csv"  # Your input file
output_file = "extracted_names_gemma.csv"  # Output file

# Read the data
data = pd.read_csv(input_file, low_memory=False).fillna('')

# Combine relevant columns to form sentences
data['sentences'] = data['Name'] + ', ' + data['Address'] + ', ' + data['City State Zip1'] + ', ' + data['City State Zip2'] + ', ' + data['PO Box'] + ', ' + data['POCity1 State Zip']
data['sentences'] = data['sentences'].str.lower()
sentences = data['sentences'].tolist()

# Process sentences in batches using Gemma
batch_size = 1  # Adjust based on memory availability
gemma_results = []

for i in range(0, len(sentences), batch_size):
    batch_sentences = sentences[i:i + batch_size]
    batch_results = extract_named_entities_batch(batch_sentences)
    gemma_results.extend(batch_results)

# Save Gemma results to CSV
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Record Number", "Original Sentence", "Extracted Names (Gemma)"])

    for i, (sentence, gemma_name) in enumerate(zip(sentences, gemma_results), start=1):
        writer.writerow([i, sentence, gemma_name])

print(f"\n✅ Extraction complete! Results saved to '{output_file}'")



✅ Extraction complete! Results saved to 'extracted_names_gemma.csv'


In [41]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load pre-trained RoBERTa model and tokenizer
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# If you have GPU available, move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Load Dataset
df = pd.read_csv("luma.csv")
df = df[['title', 'description']].dropna()  # Drop rows with NaN in descriptions

# Ensure descriptions are strings
df['description'] = df['description'].astype(str)

# Function to Generate Embeddings (batch processing)
def get_embedding(texts):
    # Tokenize all texts at once to avoid repetitive calls
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    # Extract the [CLS] token embeddings and return as a numpy array
    embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # Move to CPU
    return embeddings

# Compute Embeddings for All Movies
df['embedding'] = list(get_embedding(df['description'].tolist()))  # Convert descriptions into embeddings

# Function to Recommend Movies
def recommend_movies(user_input, top_n=5):
    user_embedding = get_embedding([user_input])  # Get the user input embedding (make it a list)
    similarities = [cosine_similarity(user_embedding, emb.reshape(1, -1))[0][0] for emb in df['embedding']]
    df['similarity'] = similarities
    recommendations = df.sort_values(by='similarity', ascending=False).head(top_n)
    return recommendations[['title', 'description', 'similarity']]

# Example Run
query = "I love thrilling action movies set in space, with a comedic twist."
print(recommend_movies(query))


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 3260: invalid start byte

In [33]:
model = model.to("cuda")
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to("cuda")


NameError: name 'text' is not defined

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load Dataset
df = pd.read_csv("luma1.csv")  # Replace with your dataset
pd.set_option('display.max_colwidth', None)
df = df[['title', 'description']].dropna()  # Drop rows with NaN in descriptions

# Initialize the TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the descriptions to obtain the TF-IDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(df['description'])

# Function to Recommend Movies
def recommend_movies(user_input, top_n=5):
    # Transform the user input to the same TF-IDF space
    user_tfidf = tfidf_vectorizer.transform([user_input])

    # Compute cosine similarity between the user input and all movie descriptions
    cosine_similarities = cosine_similarity(user_tfidf, tfidf_matrix)

    # Get the top N most similar movies
    similar_indices = cosine_similarities[0].argsort()[-top_n:][::-1]

    recommendations = df.iloc[similar_indices]
    recommendations['similarity'] = cosine_similarities[0][similar_indices]

    return recommendations[['title', 'description', 'similarity']]

# Example Run
query = "I love thrilling action movies with a space twist."
print(recommend_movies(query))


                                        title  \
72                A StoryBots Space Adventure   
7                     Guardians of the Galaxy   
103  Countdown: Inspiration4 Mission to Space   
189         The Ingenuity of the Househusband   
166                               Mystery Men   

                                                                                                                                                                                                                                                     description  \
72                                                                                                                    Join the StoryBots and the space travelers of the historic Inspiration4 mission as they search for answers to kids' questions about space.   
7    An action packed, epic space adventure, Marvel's Guardians of the Galaxy, expands the Marvel Cinematic Universe into the cosmos, where brash adventurer Peter Quill finds himself th

<ipython-input-1-6056e1f59239>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommendations['similarity'] = cosine_similarities[0][similar_indices]


In [27]:
# Install the correct modules
!pip install pandas scikit-learn transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 775.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit